# Networking (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

Installing Packages Expecto

Installing Packages Expecto.

Installing Packages Expecto..

Installing Packages Expecto...

Installing Packages Expecto....

Installing Packages Expecto.....

Installing Packages Expecto......

Installed Packages Expecto, 10.1.0

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs

In [ ]:
open Common

## testPortOpen

In [ ]:
let testPortOpen port = async {
    let! ct = Async.CancellationToken
    use client = new System.Net.Sockets.TcpClient ()
    try
        do! client.ConnectAsync ("127.0.0.1", port, ct) |> Async.awaitValueTaskUnit
        return true
    with ex ->
        trace Verbose (fun () -> $"testPortOpen / ex: {ex |> printException}") getLocals
        return false
}

In [ ]:
//// test

testPortOpen 65536
|> Async.runWithTimeout 60
|> _equal (Some false)

23:37:51 #1 [Verbose] testPortOpen / ex: System.ArgumentOutOfRangeException: Specified argument was out of the range of valid values. (Parameter 'port')
FSharpOption<Boolean>
      Value: False


In [ ]:
let testPortOpenTimeout timeout port = async {
    let! result =
        testPortOpen port
        |> Async.runWithTimeoutAsync timeout
    return
        match result with
        | None -> false
        | Some result -> result
}

In [ ]:
//// test

testPortOpenTimeout 60 65535
|> Async.RunSynchronously
|> _equal false

23:37:51 #2 [Debug] runWithTimeoutAsync / timeout: 60
False


## waitForPortAccess

In [ ]:
let waitForPortAccess timeout status port =
    let rec loop retry = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if isPortOpen = status
        then return retry
        else
            if retry % 100 = 0 then
                let getLocals () = $"port: {port} / retry: {retry} / {getLocals ()}"
                trace Warn (fun () -> "waitForPortAccess") getLocals
            do! Async.Sleep 1
            return! loop (retry + 1)
    }
    loop 0

In [ ]:
//// test

let port = 5000

let lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 5000
    let listener = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 2000
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! _ = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess None true port
    let! retries2 = waitForPortAccess None false port
    return retries1, retries2
}
|> Async.runWithTimeout 20000
|> function
    | Some (retries1, retries2) ->
        retries1 |> _isBetween 2 600
        retries2 |> _isBetween 80 250
        true
    | _ -> false
|> _equal true

23:37:51 #3 [Debug] 1
23:37:51 #4 [Debug] _1
23:37:51 #5 [Debug] 2
23:37:51 #6 [Debug] 3
23:37:53 #7 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
23:37:53 #8 [Warn] waitForPortAccess / port: 5000 / retry: 0
23:37:55 #9 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
23:37:56 #10 [Debug] _2
23:37:56 #11 [Debug] _3
23:37:56 #12 [Warn] waitForPortAccess / port: 5000 / retry: 0
23:37:58 #13 [Warn] waitForPortAccess / port: 5000 / retry: 100
23:37:58 #14 [Debug] _4
23:37:58 #15 [Debug] _5
23:38:00 #16 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
2
2
2
128
128
128
True


In [ ]:
//// test

let port = 5000

let lockPort () = async {
    trace Debug (fun () -> "_1") getLocals
    do! Async.Sleep 500
    let listener = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    trace Debug (fun () -> "_2") getLocals
    listener.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 200
    trace Debug (fun () -> "_4") getLocals
    listener.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! _ = lockPort () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 1
    trace Debug (fun () -> "3") getLocals
    let! retries1 = waitForPortAccess (Some 60) true port
    let! retries2 = waitForPortAccess (Some 60) false port
    return retries1, retries2
}
|> Async.runWithTimeout 2000
|> function
    | Some (retries1, retries2) ->
        retries1 |> _isBetween 2 60
        retries2 |> _isBetween 8 25
        true
    | _ -> false
|> _equal true

23:38:00 #17 [Debug] 1
23:38:00 #19 [Debug] _1
23:38:00 #18 [Debug] 2
23:38:00 #20 [Debug] 3
23:38:00 #21 [Debug] runWithTimeoutAsync / timeout: 60
23:38:00 #22 [Warn] waitForPortAccess / port: 5000 / retry: 0
23:38:00 #23 [Debug] runWithTimeoutAsync / timeout: 60
23:38:01 #24 [Debug] runWithTimeoutAsync / timeout: 60
23:38:01 #25 [Debug] runWithTimeoutAsync / timeout: 60
23:38:01 #26 [Debug] runWithTimeoutAsync / timeout: 60
23:38:01 #27 [Debug] runWithTimeoutAsync / timeout: 60
23:38:01 #28 [Debug] _2
23:38:01 #29 [Debug] _3
23:38:01 #30 [Debug] runWithTimeoutAsync / timeout: 60
23:38:01 #31 [Warn] waitForPortAccess / port: 5000 / retry: 0
23:38:01 #32 [Debug] _4
23:38:01 #33 [Debug] _5
23:38:01 #34 [Debug] runWithTimeoutAsync / timeout: 60
7
7
7
12
12
12
True


## getAvailablePort

In [ ]:
let getAvailablePort timeout initialPort =
    let rec loop port = async {
        let! isPortOpen =
            match timeout with
            | None -> testPortOpen port
            | Some timeout -> testPortOpenTimeout timeout port
        if not isPortOpen
        then return port
        else return! loop (port + 1)
    }
    loop initialPort

In [ ]:
//// test

let port = 5000

let lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    let listener1 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    let listener2 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 4000
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! _ = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 60
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort None port
    let! retries = waitForPortAccess None false port
    return availablePort, retries
}
|> Async.runWithTimeout 15000
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)
        retries |> _isBetween 100 400
        true
    | _ -> false
|> _equal true

23:38:01 #35 [Debug] 1
23:38:01 #36 [Debug] 2
23:38:01 #37 [Debug] _1
23:38:01 #38 [Debug] _2
23:38:01 #39 [Debug] _3
23:38:01 #40 [Debug] 3
23:38:03 #41 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
23:38:03 #42 [Warn] waitForPortAccess / port: 5000 / retry: 0
23:38:05 #43 [Warn] waitForPortAccess / port: 5000 / retry: 100
23:38:05 #44 [Debug] _4
23:38:05 #45 [Debug] _5
23:38:07 #46 [Verbose] testPortOpen / ex: System.AggregateException: One or more errors occurred. (No connection could be made because the target machine actively refused it.)
5002
125
125
125
True


In [ ]:
//// test

let port = 5000

let lockPorts () = async {
    trace Debug (fun () -> "_1") getLocals
    let listener1 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port)
    let listener2 = System.Net.Sockets.TcpListener (System.Net.IPAddress.Parse "127.0.0.1", port + 1)
    trace Debug (fun () -> "_2") getLocals
    listener1.Start ()
    listener2.Start ()
    trace Debug (fun () -> "_3") getLocals
    do! Async.Sleep 400
    trace Debug (fun () -> "_4") getLocals
    listener1.Stop ()
    listener2.Stop ()
    trace Debug (fun () -> "_5") getLocals
}

async {
    trace Debug (fun () -> "1") getLocals
    let! _ = lockPorts () |> Async.StartChild
    trace Debug (fun () -> "2") getLocals
    do! Async.Sleep 20
    trace Debug (fun () -> "3") getLocals
    let! availablePort = getAvailablePort (Some 60) port
    let! retries = waitForPortAccess (Some 60) false port
    return availablePort, retries
}
|> Async.runWithTimeout 1500
|> function
    | Some (availablePort, retries) ->
        availablePort |> _equal (port + 2)
        retries |> _isBetween 10 40
        true
    | _ -> false
|> _equal true

23:38:07 #47 [Debug] 1
23:38:07 #48 [Debug] 2
23:38:07 #49 [Debug] _1
23:38:07 #50 [Debug] _2
23:38:07 #51 [Debug] _3
23:38:08 #52 [Debug] 3
23:38:08 #53 [Debug] runWithTimeoutAsync / timeout: 60
23:38:08 #54 [Warn] waitForPortAccess / port: 5000 / retry: 0
23:38:08 #55 [Debug] _4
23:38:08 #56 [Debug] _5
23:38:08 #57 [Debug] runWithTimeoutAsync / timeout: 60
5002
21
21
21
True
